<a href="https://colab.research.google.com/github/munera72/AI-Kaggle-Project/blob/main/02%20-%20preprocesado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *PREPROCESADO PROYECTO KAGGLE*

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


## download data directly from Kaggle

- create a file `kaggle.json` with your authentication token (in kaggle $\to$ click user icon on top-right $\to$ settings $\to$ API create new token)
- upload it to this notebook workspace
- run the following cell

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 89% 18.0M/20.1M [00:01<00:00, 17.8MB/s]
100% 20.1M/20.1M [00:01<00:00, 12.9MB/s]


## unzip data

In [ ]:
!unzip udea*.zip > /dev/null

replace submission_example.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  50135751 test.csv
   692501  10666231 118025055 train.csv
  1286075  15528571 172877479 total


## load `train.csv` data with pandas



In [ ]:
import pandas as pd
import numpy as np

z = pd.read_csv("train.csv")
print ("shape of loaded dataframe", z.shape)


shape of loaded dataframe (692500, 12)


In [ ]:
z.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


In [ ]:
z = z[['FAMI_ESTRATOVIVIENDA', 'ESTU_HORASSEMANATRABAJA', 'RENDIMIENTO_GLOBAL']]
z.head()

,FAMI_ESTRATOVIVIENDA,ESTU_HORASSEMANATRABAJA,RENDIMIENTO_GLOBAL
0,Estrato 3,Menos de 10 horas,medio-alto
1,Estrato 3,0,bajo
2,Estrato 3,Más de 30 horas,bajo
3,Estrato 4,0,alto
4,Estrato 3,Entre 21 y 30 horas,medio-bajo


## Basic cleanup


In [ ]:
sum(z.ESTU_HORASSEMANATRABAJA.isna()), sum(z.FAMI_ESTRATOVIVIENDA.isna())

(30857, 32137)

we will substitute `nan` values with a preset symbol to mark them

In [ ]:
z.ESTU_HORASSEMANATRABAJA.values[z.ESTU_HORASSEMANATRABAJA.isna()] = 'no info'
z.FAMI_ESTRATOVIVIENDA.values[z.FAMI_ESTRATOVIVIENDA.isna()] = 'no info'
sum(z.ESTU_HORASSEMANATRABAJA.isna()), sum(z.FAMI_ESTRATOVIVIENDA.isna())

(0, 0)

### Cleaning `ESTU_HORASSEMANATRABAJA`

we convert `ESTU_HORASSEMANATRABAJA` to a **continuous** value since the ordering makes sense and we would like models to have a chance to capture it.

observe we assign `no info` to -1, which is somewhat artificial in this case

In [ ]:
cmap = {'Entre 11 y 20 horas': 15.5,
 'Entre 21 y 30 horas': 25.5,
 'Menos de 10 horas': 5,
 'Más de 30 horas': 35,
 '0': 0,
 'no info': -1}

z.ESTU_HORASSEMANATRABAJA = np.r_[[cmap[i] for i in z.ESTU_HORASSEMANATRABAJA]]
z.ESTU_HORASSEMANATRABAJA.value_counts()

,count
ESTU_HORASSEMANATRABAJA,
35.0,249352
0.0,116550
15.5,115857
25.5,92693
5.0,87191
-1.0,30857


### Cleaning `FAMI_ESTRATOVIVIENDA`



In [ ]:
cmap = {'Estrato 1': 1,
 'Estrato 2': 2,
 'Estrato 3': 3,
 'Estrato 4': 4,
 'Estrato 5': 5,
 'Estrato 6': 6,
 'Sin Estrato': 0,
 'no info': -1}

z.FAMI_ESTRATOVIVIENDA = np.r_[[cmap[i] for i in z.FAMI_ESTRATOVIVIENDA]]
z.FAMI_ESTRATOVIVIENDA.value_counts()

,count
FAMI_ESTRATOVIVIENDA,
2,232671
3,210685
1,111991
4,65514
-1,32137
5,23608
6,12605
0,3289


first we create the onehot mapping

In [ ]:
x = z.FAMI_ESTRATOVIVIENDA.values
FAMI_ESTRATOVIVIENDA_vals = sorted(np.unique(x))
FAMI_ESTRATOVIVIENDA_onehot_vals = {val: np.eye(len(FAMI_ESTRATOVIVIENDA_vals))[i] for i,val in enumerate(FAMI_ESTRATOVIVIENDA_vals)}
FAMI_ESTRATOVIVIENDA_onehot_vals

{-1: array([1., 0., 0., 0., 0., 0., 0., 0.]),
 0: array([0., 1., 0., 0., 0., 0., 0., 0.]),
 1: array([0., 0., 1., 0., 0., 0., 0., 0.]),
 2: array([0., 0., 0., 1., 0., 0., 0., 0.]),
 3: array([0., 0., 0., 0., 1., 0., 0., 0.]),
 4: array([0., 0., 0., 0., 0., 1., 0., 0.]),
 5: array([0., 0., 0., 0., 0., 0., 1., 0.]),
 6: array([0., 0., 0., 0., 0., 0., 0., 1.])}

In [ ]:
FAMI_ESTRATOVIVIENDA_onehot_enc = np.r_[[FAMI_ESTRATOVIVIENDA_onehot_vals[i] for i in z.FAMI_ESTRATOVIVIENDA]]
FAMI_ESTRATOVIVIENDA_onehot_enc

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]])

In [ ]:
FAMI_ESTRATOVIVIENDA_df = pd.DataFrame(FAMI_ESTRATOVIVIENDA_onehot_enc, columns=[f"FAMI_ESTRATOVIVIENDA__{v}" for v in FAMI_ESTRATOVIVIENDA_onehot_vals])
FAMI_ESTRATOVIVIENDA_df

,FAMI_ESTRATOVIVIENDA__-1,FAMI_ESTRATOVIVIENDA__0,FAMI_ESTRATOVIVIENDA__1,FAMI_ESTRATOVIVIENDA__2,FAMI_ESTRATOVIVIENDA__3,FAMI_ESTRATOVIVIENDA__4,FAMI_ESTRATOVIVIENDA__5,FAMI_ESTRATOVIVIENDA__6
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
692495,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
692496,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692497,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
692498,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


and we assemble everything into a single dataframe, removing the original `FAMI_ESTRATOVIVIENDA` column.




In [ ]:
z = pd.concat([FAMI_ESTRATOVIVIENDA_df, z], axis=1).drop('FAMI_ESTRATOVIVIENDA', axis=1)
z.shape

(692500, 10)

In [ ]:
z.head()

,FAMI_ESTRATOVIVIENDA__-1,FAMI_ESTRATOVIVIENDA__0,FAMI_ESTRATOVIVIENDA__1,FAMI_ESTRATOVIVIENDA__2,FAMI_ESTRATOVIVIENDA__3,FAMI_ESTRATOVIVIENDA__4,FAMI_ESTRATOVIVIENDA__5,FAMI_ESTRATOVIVIENDA__6,ESTU_HORASSEMANATRABAJA,RENDIMIENTO_GLOBAL
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,medio-alto
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,bajo
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,35.0,bajo
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,alto
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25.5,medio-bajo


## convert target (prediction) column into discrete values

now everything in our dataset is **numeric**!!!

In [ ]:
y_col = 'RENDIMIENTO_GLOBAL'

rmap = {'alto': 3, 'bajo':0, 'medio-bajo':1, 'medio-alto':2}
z[y_col] = [rmap[i] for i in z[y_col]]
z.head()

,FAMI_ESTRATOVIVIENDA__-1,FAMI_ESTRATOVIVIENDA__0,FAMI_ESTRATOVIVIENDA__1,FAMI_ESTRATOVIVIENDA__2,FAMI_ESTRATOVIVIENDA__3,FAMI_ESTRATOVIVIENDA__4,FAMI_ESTRATOVIVIENDA__5,FAMI_ESTRATOVIVIENDA__6,ESTU_HORASSEMANATRABAJA,RENDIMIENTO_GLOBAL
0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,5.0,2
1,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,35.0,0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,25.5,1
